In [ ]:
import torch

def print_gpu_memory(prefix=""):
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / (1024 ** 2)
        reserved = torch.cuda.memory_reserved() / (1024 ** 2)
        print(f"{prefix} Memory Allocated: {allocated:.2f} MB")
        print(f"{prefix} Memory Reserved: {reserved:.2f} MB")
    else:
        print("CUDA is not available.")


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache() 

print_gpu_memory()

device

In [ ]:
# imports

import os
import sys
sys.path.append(os.path.abspath('..'))

import src.data.preprocess_data as data
import src.training.train_model as train
import src.data.view as view
import src.models.unets as unets
import src.models.hrnets as hrnets

from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore


In [ ]:
working_dir = os.path.abspath('..')
models_dir = os.path.join(working_dir, 'models')
models_names = os.listdir(models_dir)

load_model = True
infos = {}
for model_name in models_names:
    print(model_name)
    try:
        patch_size = int(model_name.split('-')[1])
        weighted = 'W.pth' in model_name
        if model_name.split('-')[2]=='type':
            num_classes = 5
        if model_name.split('-')[2]=='binary':
            num_classes = 2
        crf = False
        dist = False
        if len(model_name.split('-'))>=4 and model_name.split('-')[3]=='crf':
            crf = True
        if len(model_name.split('-'))>=4 and model_name.split('-')[3]=='dist':
            dist = True
            
        if load_model:
            if model_name.startswith('UNetSmall-'):
                model = unets.UNetSmall(in_channels=12, out_channels=num_classes, crf=crf, use_dist=dist).to(device) 
            if model_name.startswith('UNet-'):
                model = unets.UNet(in_channels=12, out_channels=num_classes).to(device) 
            elif model_name.startswith('UNetResNet34-'):
                model = unets.UNetResNet34(in_channels=12, out_channels=num_classes).to(device) 
            elif model_name.startswith('UNetEfficientNetB0-'):
                model = unets.UNetEfficientNetB0(in_channels=12, out_channels=num_classes).to(device) 
            elif model_name.startswith('UNetConvNext-'):
                model = unets.UNetConvNext (in_channels=12, out_channels=num_classes).to(device) 
            elif model_name.startswith('HRNetW18'):
                model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w18", pretrained=True,).to(device)
            elif model_name.startswith('HRNetW32'):
                model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w32", pretrained=True,).to(device)
            elif model_name.startswith('HRNetW48'):
                model = hrnets.HRNetSegmentation(in_channels= 12, num_classes=num_classes, backbone="hrnet_w48", pretrained=True,).to(device)
        ckp_file = os.path.join(models_dir, model_name)
        checkpoint = torch.load(ckp_file, weights_only=False)
        info = checkpoint['best_epoch_info']
        infos[model_name] = info
        print(info)
    except Exception as e: 
        print('error loading.')
        print(e)    
        

In [ ]:
import pandas as pd
df = pd.DataFrame(infos).T
df = df[['epoch', 'lr', 'train_acc', 'val_acc', 'train_micro', 'val_micro', 'train_macro', 'val_macro', 'train_time', 'val_time', 'train_memory', 'val_memory']]
df = df.applymap(lambda x: x.item() if isinstance(x, torch.Tensor) else x)
dft = df[['train_time', 'val_time']].applymap(lambda x: f'{x:.2f}' if isinstance(x, (float, int)) else x)
df[['train_time', 'val_time']] = dft
df_ = df[['train_acc', 'val_acc', 'train_micro', 'val_micro', 'train_macro', 'val_macro',]].applymap(lambda x: f'{100*x:.2f}' if isinstance(x, (float, int)) else x)
df[['train_acc', 'val_acc', 'train_micro', 'val_micro', 'train_macro', 'val_macro',]] = df_
df[['lr']]= df[['lr']].applymap(lambda x: f'{x:.8f}' if isinstance(x, (float, int)) else x)
col_names = ['Melhor época', 'LR final', 'Acurácia de treino', 'Acurácia de validação', 'Micro F1-Score de treino','Micro F1-Score de validação', 
             'Macro F1-Score de treino','Macro F1-Score de validação', 'Tempo de treino (época)','Tempo de validação (época)', 'Memória de treino', 'Memória de validação']
map = {c1:c2 for c1, c2 in zip(df.columns,col_names)}
df = df.rename(columns=map)


df = df.sort_index().astype(str)

print(df.T.to_latex())
df

In [ ]:

key = ['epoch', 'lr', 'train_acc', 'val_acc', 'train_micro', 'val_micro', 'train_time', 'val_time', 'train_memory', 'val_memory']
col_names = ['Melhor época', 'LR final', 'Acurácia de treino', 'Acurácia de validação', 'Micro F1-Score de treino','Micro F1-Score de validação', 
             'Macro F1-Score de treino','Macro F1-Score de validação', 'Tempo de treino (época)','Tempo de validação (época)', 'Memória de treino', 'Memória de validação']
for model,v in infos.items():
    
    print(model)
    print([k for k in key])
    print([v[k] for k in key])

In [ ]:
import numpy as np

def classification_report_from_cm(cm, class_names=None):
    n_classes = cm.shape[0]
    report = {}
    total_samples = np.sum(cm)
    support = np.sum(cm, axis=1)  # true samples per class
    
    precisions = []
    recalls = []
    f1s = []
    
    for i in range(n_classes):
        TP = cm[i, i]
        FP = np.sum(cm[:, i]) - TP
        FN = np.sum(cm[i, :]) - TP
        # TN is not used for per-class precision/recall/F1
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0.0

        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
        
        if class_names:
            label = class_names[i]
        else:
            label = f"Class {i}"
            
        report[label] = {
            "precision": precision,
            "recall": recall,
            "f1-score": f1,
            "support": support[i]
        }
    
    # Optionally, compute weighted averages:
    weighted_precision = np.average(precisions, weights=support)
    weighted_recall = np.average(recalls, weights=support)
    weighted_f1 = np.average(f1s, weights=support)
    
    report["weighted avg"] = {
        "precision": weighted_precision,
        "recall": weighted_recall,
        "f1-score": weighted_f1,
        "support": total_samples
    }
    
    return report

# Example confusion matrix for 3 classes
cm = np.array([[50,  2,  3],
               [ 4, 45,  1],
               [ 5,  2, 40]])

report = classification_report_from_cm(cm, class_names=["A", "B", "C"])
for label, metrics in report.items():
    print(label, metrics)


In [ ]:
infos


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

infos = dict(sorted(infos.items()))
keys = ["train_f1_C0", "train_f1_C1", "train_f1_C2", "train_f1_C3", "train_f1_C4"]
names = list(infos.keys())  # X-axis labels
values = [[infos[name][k] for k in keys] for name in names]  # Data for each group
save_to = os.path.join(working_dir,"figs","train_f1.png")
view.plot_horizontal(values, names, "F1", "Treino", save_to=save_to)


In [ ]:

keys = ["val_f1_C0", "val_f1_C1", "val_f1_C2", "val_f1_C3", "val_f1_C4"]
names = list(infos.keys())  # X-axis labels
values = [[infos[name][k] for k in keys] for name in names]  # Data for each group
save_to = os.path.join(working_dir,"figs","val_f1.png")
view.plot_horizontal(values, names, "F1", "Validação", save_to=save_to)

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

infos = dict(sorted(infos.items()))



keys = ["val_f1_C0", "val_f1_C1", "val_f1_C2", "val_f1_C3", "val_f1_C4"]
keys = ["train_acc", "val_acc", "train_micro", "val_micro", "train_macro", "val_macro"]

names = list(infos.keys())  # X-axis labels
train_acc = [infos[name]['train_acc'] for name in names]
val_acc = [infos[name]['val_acc'] for name in names]
train_micro = [infos[name]['train_micro'] for name in names]
val_micro = [infos[name]['val_micro'] for name in names]
train_macro = [infos[name]['train_macro'] for name in names]
val_macro = [infos[name]['val_macro'] for name in names]

fig, ax = plt.subplots(figsize=(20, 8)) 

ax.bar(names, val_acc)
ax.bar(names, train_acc)
ax.bar(names, train_micro)
ax.bar(names, val_micro)
ax.bar(names, train_macro)
ax.bar(names, val_macro)
ax.set_xticklabels(names, rotation=90)
plt.show()

In [ ]:

keys = ["train_acc", "val_acc", "train_micro", "val_micro", "train_macro", "val_macro"]
names = list(infos.keys())  # X-axis labels
save_to = os.path.join(working_dir,"figs","train_val_metrics.png")
metric_names = ["Acurácia de treino", "Acurácia de validação", "F1-score (micro) de treino", "F1-score (micro) de validação", "F1-score (macro) de treino", "F1-score (macro) de validação"]
#view.plot_horizontal_metric_combination(infos, names, keys, metric_names=metric_names, save_to=save_to)
#view.plot_vertical_metric_combination(infos, names, keys, metric_names=metric_names, save_to=save_to)
suffixes = ['-type-CE.pth', '-type-CEW.pth', '-type-DS-CE.pth', '-type-DS-CEW.pth']
view.plot_metric_combination(infos, names, keys, suffixes, metric_names=metric_names, save_to=save_to)
print(infos, names, keys, metric_names)

In [ ]:
keys = ["train_acc", "train_micro", "train_macro", ]
names = list(infos.keys())  # X-axis labels
save_to = os.path.join(working_dir,"figs","train_metrics.png")
metric_names = ["Acurácia de treino", "F1-score (micro) de treino", "F1-score (macro) de treino",]
view.plot_horizontal_metric_combination(infos, names, keys, metric_names=metric_names, save_to=save_to)


In [ ]:
keys = ["val_acc", "val_micro", "val_macro"]
names = list(infos.keys())  # X-axis labels
save_to = os.path.join(working_dir,"figs","val_metrics.png")
metric_names = ["Acurácia de validação", "F1-score (micro) de validação", "F1-score (macro) de validação"]
view.plot_horizontal_metric_combination(infos, names, keys, metric_names=metric_names, save_to=save_to)


In [ ]:
#info['val_report']['precision']
#info['val_report']['recall']
precisions = []
recalls = []
precisions_train = []
recalls_train = []

#values = [[infos[name][k] for k in keys] for name in names]  # Data for each group

for k,v in infos.items():
    print(k)
    prec=[]
    rec = []
    prec_t=[]
    rec_t = []
    for i in range(5):
        prec.append(v['val_report']['precision'][f'Class {i}'])
        rec.append(v['val_report']['recall'][f'Class {i}'])
        prec_t.append(v['train_report']['precision'][f'Class {i}'])
        rec_t.append(v['train_report']['recall'][f'Class {i}'])
        #print(v['val_report']['precision'][f'Class {i}'])
        #print(v['val_report']['recall'][f'Class {i}'])
    precisions.append(prec)
    recalls.append(rec)
    precisions_train.append(prec_t)
    recalls_train.append(rec_t)


In [ ]:
save_to = os.path.join(working_dir,"figs","train_precision.png")
view.plot_horizontal(precisions_train, names, "Precisão", "Treino", save_to=save_to)
view.plot_vertical(precisions_train, names, "Precisão", "Treino", save_to=save_to)

In [ ]:
save_to = os.path.join(working_dir,"figs","val_precision.png")
view.plot_horizontal(precisions, names, "Precisão", "Validação", save_to=save_to)

In [ ]:
save_to = os.path.join(working_dir,"figs","train_recall.png")
view.plot_horizontal(recalls_train, names, "Recall", "Treino", save_to=save_to)


In [ ]:
save_to = os.path.join(working_dir,"figs","val_recall.png")
view.plot_horizontal(recalls, names, "Recall", "Validação")


In [ ]:
suffixes = ['-type-CE.pth', '-type-CEW.pth', '-type-DS-CE.pth', '-type-DS-CEW.pth']
save_to = os.path.join(working_dir,"figs","train_recall.png")
view.plot_metric(recalls_train, names, suffixes, metric_name="Recall", save_to=save_to)
save_to = os.path.join(working_dir,"figs","val_recall.png")
view.plot_metric(recalls, names, suffixes, metric_name="Recall", save_to=save_to)


In [ ]:
Classes:
0 -> fundo
1 -> loteamento vazio
2 -> outros equipamentos
3 -> vazio intraurbano
4 -> area urbanizada

LR dos HRNet muito baixo? mais epocas/paciencia?

weighted: ajudou na classe 3


In [ ]:
infos